In [2]:
#Libraries
import mysql.connector
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()


In [ ]:
database = "SD_PD"
user = 'Salvador'
password = 'renovatioA19!'
host = 'localhost'

In [ ]:
#Initial Connection
cnx = mysql.connector.connect(user=user,
                              password=password,
                              host=host,
                              database=database
                              )
cursor = cnx.cursor()

In [ ]:
#Calls
url = f"http://seshat.datasd.org/pd/pd_cfs_calltypes_datasd.csv"
call_type = pd.read_csv(url).dropna(axis=1)


In [ ]:
#Dispos
url = f"http://seshat.datasd.org/pd/pd_dispo_codes_datasd.csv"
disposition = pd.read_csv(url).dropna(axis=1)
disposition.loc[len(disposition.index)] = ['DEF', 'DEF'] 
disposition.loc[len(disposition.index)] = ['SS-', 'SS-'] 

In [ ]:
#Beats
beat = pd.read_csv('beat-neighborhood.csv').dropna(axis=1)
beat_split = beat['beat,neighborhood'].str.split(pat=",",
                                                 expand=True)
beat_split = beat_split.rename(columns={0: "beat",
                                        1: "neighborhood"})
beat_split.loc[len(beat_split.index)] = [-1, 'No Beat'] 


In [ ]:
# Create dataframe
days = pd.DataFrame({
    'day_of_week':list(range(1,8)),
    'day':['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'],
})


In [ ]:
# df to dict
call_type_dict = call_type.to_dict('records')
disposition_dict = disposition.to_dict('records')
beat_dict = beat_split.to_dict('records')
days_dict = days.to_dict('records')

In [ ]:
def Find_Data(row,cursor,table, key):
    try:
        #print(f"SELECT * from {table} where {key} = '{row[key]}'")
        cursor.execute(f"SELECT * from {table} where {key} = '{row[key]}'")
        # Fetch a record
        result = cursor.fetchone()
    except:
        result = 0
    
    return result

In [ ]:
#Load Data
def Load_Data(row,cursor,table,Col1,Col2):
    # Create a new record
    sql = f"""INSERT INTO {table} ({Col1},
                                    {Col2}
                                    ) VALUES (%s, %s)"""

    # Execute the query
    cursor.execute(sql, (row[Col1],row[Col2]))
    cnx.commit()  

In [ ]:
# SQL Variable Tables
SQL_Tables = pd.DataFrame({
    'Table':['Call_types','Dispositions','Beats'],
    'Key':['call_type','dispo_code','beat'],
    'Col2':['description','description','neighborhood'],
    'dict':[call_type_dict,disposition_dict,beat_dict]
})
SQL_Tables_dict = SQL_Tables.to_dict('records')

In [ ]:
for upper_row in tqdm(SQL_Tables_dict):
    for row in upper_row['dict']:
        Data_Found = Find_Data(row,cursor,
                               upper_row['Table'],
                               upper_row['Key'])
        if Data_Found is None:
            #If no data found load 
            Load_Data(row,
                      cursor,
                      upper_row['Table'],
                      upper_row['Key'],
                      upper_row['Col2'])
        else:
            pass

# Load Historical Calls

In [ ]:
#function to download yearly data
def Get_Data(year):
    url = f"https://seshat.datasd.org/pd/pd_calls_for_service_{year}_datasd.csv"
    df = pd.read_csv(url)
    return df

In [ ]:
#Calls
def Load_Calls(row,cursor,table,year):
    # Create a new record
    sql = f"""INSERT INTO {table} ( incident_num,
                                    incident_date_time,
                                    call_type,
                                    dispo_code,
                                    beat,
                                    priority,
                                    incident_year
                                    ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""

    # Execute the query
    cursor.execute(sql, (row['incident_num'],
                         row['date_time'],
                         row['call_type'],
                         row['disposition'],
                         row['beat'],
                         row['priority'],
                         year,
                        )
                  )
    cnx.commit()  

In [ ]:
#Calls
def Load_Calls_Address(row,cursor,table):
    address = ' '.join(filter(None, (str(row['address_number_primary_str']),
                                  row['address_dir_primary'],
                                  row['address_road_primary'],
                                  row['address_sfx_primary'],
                                  'San Diego CA'
                                 )))

    # Create a new record
    sql = f"""INSERT INTO {table} ( incident_num,
                                    address_number_primary,
                                    address_dir_primary,
                                    address_road_primary,
                                    address_sfx_primary,
                                    address_dir_intersecting,
                                    address_road_intersecting,
                                    address_sfx_intersecting,
                                    address
                                    ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"""

    # Execute the query
    cursor.execute(sql, (row['incident_num'],
                         row['address_number_primary'],
                         row['address_dir_primary'],
                         row['address_road_primary'],
                         row['address_sfx_primary'],
                         row['address_dir_intersecting'],
                         row['address_road_intersecting'],
                         row['address_sfx_intersecting'],
                         row['address_road_intersecting'],
                         address
                        )
                  )
    cnx.commit()

In [ ]:
#Calls
def Load_Calls_Address(row,cursor,table):
    address = ' '.join(filter(None, (str(row['address_number_primary_str']),
                                  row['address_dir_primary'],
                                  row['address_road_primary'],
                                  row['address_sfx_primary'],
                                  'San Diego CA'
                                 )))
    print(address)


In [4]:
Years = list(range(2023,datetime.now().year +1))
Years

[2023]

In [ ]:
for year in Years:
    yearly_data = Get_Data(year)
    yearly_data_dict = yearly_data.to_dict('records')
    for row in tqdm(yearly_data_dict):
        #Load Service Calls
        Table = 'Service_calls'
        Key = 'incident_num'
        #Verify if Data is in table
        Data_Found = Find_Data(row,cursor,Table,Key)
        if Data_Found is None:
            #If no data found load
            try:
                Load_Calls(row,cursor,Table,year)
            except:
                print(row)
                
        #Load Service Calls Address        
        Table = 'Address'
        Key = 'incident_num'
        #Verify if Data is in table
        Data_Found = Find_Data(row,cursor,Table,Key)
        if Data_Found is None:
            #If no data found load
            try:
                Load_Calls_Address(row,cursor,Table)
            except:
                print(row)
                

        else:
            print('error')
            print(row)